In [1]:
%cd /content
!git clone https://github.com/MioAyanokouji/FunSearch-TSP.git
%cd /content/FunSearch-TSP

/content
Cloning into 'FunSearch-TSP'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 175 (delta 10), reused 172 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (175/175), 2.13 MiB | 4.33 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/FunSearch-TSP


## 1-DataSet

In [1]:
!pip install tsplib95

In [73]:
import tsplib95
import pandas as pd
import matplotlib.pyplot as plt

import glob, os

import numpy as np
from itertools import chain
from scipy.spatial.distance import squareform, pdist
from functools import partial
import math

from typing import List

import random

In [3]:
# edge_weight_type = []
# edge_weight_format = []

# for file in glob.glob("./data/*.tsp"):
#     problem = tsplib95.load(file)
#     if problem.edge_weight_type == 'EXPLICIT':
#         edge_weight_format.append(problem.edge_weight_format)
#     else:
#         edge_weight_type.append(problem.edge_weight_type)

# edge_weight_type = list(set(edge_weight_type))
# edge_weight_format = list(set(edge_weight_format))

# print(edge_weight_type)
# print(edge_weight_format)

['ATT', 'GEO', 'EUC_2D', 'CEIL_2D']
['LOWER_DIAG_ROW', 'UPPER_ROW', 'FULL_MATRIX', 'UPPER_DIAG_ROW']


In [150]:
class TSPProblem:
    def __init__(self, problem_path=None):
        problem = tsplib95.load(problem_path)
        self.name = problem.name
        self.dimension = problem.dimension

        if problem.edge_weight_type == 'EXPLICIT':
            # nodes
            self.nodes = pd.DataFrame.from_dict(problem.display_data, orient='index', columns=['x', 'y'])

            # distance matrix
            if problem.edge_weight_format == 'FULL_MATRIX':
                self.distance_matrix = problem.edge_weights
            else:
                if problem.edge_weight_format == 'LOWER_DIAG_ROW':
                    idx = np.tril_indices(self.dimension)
                if problem.edge_weight_format == 'UPPER_ROW':
                    idx = np.triu_indices(self.dimension, k=1)
                if problem.edge_weight_format == 'UPPER_DIAG_ROW':
                    idx = np.triu_indices(self.dimension)

                distance = list(chain(*problem.edge_weights))
                self.distance_matrix = np.zeros((self.dimension, self.dimension))
                self.distance_matrix[idx] = distance
                self.distance_matrix[idx[1], idx[0]] = distance
        else:
            # nodes
            self.nodes = pd.DataFrame.from_dict(problem.node_coords, orient='index', columns=['x', 'y'])

            # distance matrix
            if problem.edge_weight_type == 'EUC_2D':
                self.distance_matrix = squareform(pdist(self.nodes, tsplib95.distances.euclidean))
            if problem.edge_weight_type == 'CEIL_2D':
                self.distance_matrix = squareform(pdist(self.nodes, partial(tsplib95.distances.euclidean, round=math.ceil)))
            if problem.edge_weight_type == 'GEO':
                self.distance_matrix = squareform(pdist(self.nodes, tsplib95.distances.geographical))
            if problem.edge_weight_type == 'ATT':
                self.distance_matrix = squareform(pdist(self.nodes, tsplib95.distances.pseudo_euclidean))

        self.distance_matrix = pd.DataFrame(self.distance_matrix, index=range(1, self.dimension+1), columns=range(1, self.dimension+1))

        # opt
        opt_path = os.path.splitext(problem_path)[0] + '.opt.tour'
        if os.path.isfile(opt_path):
            self.optimal_tour = opt.tours[0]
        else:
            self.optimal_tour = None

    def get_tour_length(self, tour: List[int]) -> float:
        length = 0
        for i in range(len(tour)):
            length += self.distance_matrix[tour[i]][tour[(i+1)%len(tour)]]
        return length

    def plot_tour(self, tour: List[int]):
        plt.figure(figsize=(10, 6))
        
        # nodes
        x = self.nodes.x
        y = self.nodes.y
        plt.scatter(x, y, color='#E53528', zorder=2)
    
        # annotation
        for i in range(self.dimension):
            plt.annotate(str(i+1), (x[i+1], y[i+1]), fontsize=6, xytext=(2, 2), textcoords='offset points')
    
        # tour
        for i in range(len(tour)):
            start = tour[i]
            end = tour[(i+1)%len(tour)]
            plt.plot([x[start], x[end]], [y[start], y[end]], color='#55B7E6', zorder=1)
    
        plt.title(f'{self.name} - Length: {self.get_tour_length(tour):.2f}')
        plt.show()

In [192]:
a280 = TSPProblem('./data/a280.tsp')

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [193]:
a280.nodes

,x,y
1,15440.0,11286.0
2,16688.0,8228.0
3,17936.0,6116.0
4,6320.0,12144.0
5,6400.0,9680.0
...,...,...
1744,13744.0,4576.0
1745,1136.0,3960.0
1746,2544.0,3960.0
1747,10128.0,4576.0


In [194]:
a280.distance_matrix

,1,2,3,4,5,6,7,8,9,10,...,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748
1,0.0,3303.0,5741.0,9160.0,9182.0,10002.0,14258.0,15174.0,15359.0,11229.0,...,15674.0,14072.0,4644.0,8156.0,7919.0,6921.0,16071.0,14832.0,8558.0,8347.0
2,3303.0,0.0,2453.0,11083.0,10390.0,10302.0,15569.0,15538.0,15551.0,13115.0,...,15695.0,16237.0,6714.0,4865.0,6027.0,4691.0,16127.0,14774.0,7508.0,6764.0
3,5741.0,2453.0,0.0,13087.0,12074.0,11479.0,17172.0,16568.0,16469.0,15062.0,...,16494.0,18276.0,8565.0,2415.0,5911.0,4466.0,16938.0,15542.0,7958.0,6844.0
4,9160.0,11083.0,13087.0,0.0,2465.0,5304.0,5484.0,8211.0,8777.0,2071.0,...,9456.0,5190.0,12573.0,15112.0,10214.0,10601.0,9688.0,9013.0,8472.0,9654.0
5,9182.0,10390.0,12074.0,2465.0,0.0,2839.0,5185.0,6480.0,6904.0,3495.0,...,7477.0,6790.0,13211.0,13859.0,8313.0,8943.0,7774.0,6898.0,6321.0,7624.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1744,6921.0,4691.0,4466.0,10601.0,8943.0,7609.0,13676.0,12409.0,12226.0,12319.0,...,12176.0,15645.0,11203.0,5316.0,1449.0,0.0,12623.0,11217.0,3616.0,2385.0
1745,16071.0,16127.0,16938.0,9688.0,7774.0,6072.0,6139.0,1646.0,920.0,9053.0,...,448.0,10711.0,20511.0,17888.0,11296.0,12623.0,0.0,1408.0,9013.0,10304.0
1746,14832.0,14774.0,15542.0,9013.0,6898.0,4878.0,6277.0,2004.0,1322.0,8666.0,...,960.0,10720.0,19324.0,16480.0,9888.0,11217.0,1408.0,0.0,7609.0,8896.0
1747,8558.0,7508.0,7958.0,8472.0,6321.0,4294.0,10470.0,8810.0,8611.0,9814.0,...,8566.0,13087.0,13190.0,8917.0,2385.0,3616.0,9013.0,7609.0,0.0,1449.0


In [196]:
a280.plot_tour(a280.optimal_tour)

TypeError: object of type 'NoneType' has no len()

In [160]:
# random.seed(0)
# dataset_path = random.sample(glob.glob("./data/*.tsp"), k=5)

dataset_path = ['./data/a280.tsp', './data/att48.tsp', './data/bayg29.tsp', './data/bays29.tsp', './data/berlin52.tsp']

dataset = {}
for path in dataset_path:
    problem = TSPProblem(path)
    dataset[problem.name] = problem.distance_matrix

## 2-LLM

In [11]:
# !pip install absl-py

In [12]:
import time
import json
import multiprocessing
from typing import Collection, Any
import http.client
from implementation import sampler

In [13]:
api_key = 'sk-0ivLxCmk7HNR86iH81Aa4dC92aA34439AbE5F857BeC1F433'

In [14]:
def _trim_preface_of_body(sample: str) -> str:
    """Trim the redundant descriptions/symbols/'def' declaration before the function body.
    Please see my comments in sampler.LLM (in sampler.py).
    Since the LLM used in this file is not a pure code completion LLM, this trim function is required.

    -Example sample (function & description generated by LLM):
    -------------------------------------
    This is the optimized function ...
    def priority_v2(...) -> ...:
        return ...
    This function aims to ...
    -------------------------------------
    -This function removes the description above the function's signature, and the function's signature.
    -The indent of the code is preserved.
    -Return of this function:
    -------------------------------------
        return ...
    This function aims to ...
    -------------------------------------
    """
    lines = sample.splitlines()
    func_body_lineno = 0
    find_def_declaration = False
    for lineno, line in enumerate(lines):
        # find the first 'def' statement in the given code
        if line[:3] == 'def':
            func_body_lineno = lineno
            find_def_declaration = True
            break
    if find_def_declaration:
        code = ''
        for line in lines[func_body_lineno + 1:]:
            code += line + '\n'
        return code
    return sample

class LLMAPI(sampler.LLM):
    """Language model that predicts continuation of provided source code.
    """

    def __init__(self, samples_per_prompt: int, trim=True):
        super().__init__(samples_per_prompt)
        additional_prompt = ('Complete a different and more complex Python function. '
                             'Be creative and you can insert multiple if-else and for-loop in the code logic.'
                             'Only output the Python code, no descriptions.')
        self._additional_prompt = additional_prompt
        self._trim = trim

    def draw_samples(self, prompt: str) -> Collection[str]:
        """Returns multiple predicted continuations of `prompt`."""
        return [self._draw_sample(prompt) for _ in range(self._samples_per_prompt)]

    def _draw_sample(self, content: str) -> str:
        prompt = '\n'.join([content, self._additional_prompt])
        while True:
            try:
                conn = http.client.HTTPSConnection("api.bltcy.ai")
                payload = json.dumps({
                    "max_tokens": 512,
                    "model": "gpt-3.5-turbo",
                    "messages": [
                        {
                            "role": "user",
                            "content": prompt
                        }
                    ]
                })
                headers = {
                    'Authorization': 'Bearer {api_key}'.format(api_key=api_key),
                    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
                    'Content-Type': 'application/json'
                }
                conn.request("POST", "/v1/chat/completions", payload, headers)
                res = conn.getresponse()
                data = res.read().decode("utf-8")
                data = json.loads(data)
                response = data['choices'][0]['message']['content']
                # trim function
                if self._trim:
                    response = _trim_preface_of_body(response)
                return response
            except Exception:
                time.sleep(2)
                continue

## 3-SandBox

In [15]:
from implementation import evaluator
from implementation import evaluator_accelerate

In [16]:
class Sandbox(evaluator.Sandbox):
    """Sandbox for executing generated code. Implemented by RZ.

    RZ: Sandbox returns the 'score' of the program and:
    1) avoids the generated code to be harmful (accessing the internet, take up too much RAM).
    2) stops the execution of the code in time (avoid endless loop).
    """

    def __init__(self, verbose=False, numba_accelerate=True):
        """
        Args:
            verbose         : Print evaluate information.
            numba_accelerate: Use numba to accelerate the evaluation. It should be noted that not all numpy functions
                              support numba acceleration, such as np.piecewise().
        """
        self._verbose = verbose
        self._numba_accelerate = numba_accelerate

    def run(
            self,
            program: str,
            function_to_run: str,  # RZ: refers to the name of the function to run (e.g., 'evaluate')
            function_to_evolve: str,  # RZ: accelerate the code by decorating @numba.jit() on function_to_evolve.
            inputs: Any,  # refers to the dataset
            test_input: str,  # refers to the current instance
            timeout_seconds: int,
            **kwargs  # RZ: add this
    ) -> tuple[Any, bool]:
        """Returns `function_to_run(test_input)` and whether execution succeeded.

        RZ: If the generated code (generated by LLM) is executed successfully,
        the output of this function is the score of a given program.
        RZ: PLEASE NOTE THAT this SandBox is only designed for bin-packing problem.
        """
        try:
            result_queue = multiprocessing.Queue()
            process = multiprocessing.Process(
                target=self._compile_and_run_function,
                args=(program, function_to_run, function_to_evolve, dataset, self._numba_accelerate, result_queue)
            )
            process.start()
            process.join(timeout=timeout_seconds)
            if process.is_alive():
                # if the process is not finished in time, we consider the program illegal
                process.terminate()
                process.join()
                results = None, False
            else:
                if not result_queue.empty():
                    results = result_queue.get_nowait()
                else:
                    results = None, False

            return results
        except:
            return None, False

    def _compile_and_run_function(self, program, function_to_run, function_to_evolve, dataset, numba_accelerate,
                                  result_queue):
        try:
            # optimize the code (decorate function_to_run with @numba.jit())
            if numba_accelerate:
                program = evaluator_accelerate.add_numba_decorator(
                    program=program,
                    function_to_evolve=function_to_evolve
                )
            # compile the program, and maps the global func/var/class name to its address
            all_globals_namespace = {}
            # execute the program, map func/var/class to global namespace
            exec(program, all_globals_namespace)
            # get the pointer of 'function_to_run'
            function_to_run = all_globals_namespace[function_to_run]
            # return the execution results
            results = function_to_run(dataset)
            # the results must be int or float
            if not isinstance(results, (int, float)):
                result_queue.put((None, False))
                return
            result_queue.put((results, True))
        except:
            # if raise any exception, we assume the execution failed
            result_queue.put((None, False))

## 4-Specification

In [189]:
import numpy as np

def tsp(distance_matrix):
    """greedy algorithm"""
    num_nodes = len(distance_matrix)
    current_node = 1
    tour = [current_node]
    total_distance = 0

    while(len(tour) < num_nodes):
        priorities = priority(distance_matrix, tour)
        next_node = int(np.argmax(priorities) + 1)
        total_distance -= priorities[next_node]
        tour.append(next_node)

    total_distance += distance_matrix[tour[-1]][1]
    return tour, total_distance

# @funsearch.run
def evaluate(distance_matrix):
    """evaluate tsp function by total distance"""
    tour, total_distance = tsp(distance_matrix)
    return -total_distance

# @funsearch.evolve
def priority(distance_matrix, tour):
    """return priorities for next node with distance"""
    num_nodes = len(distance_matrix)
    current_node = tour[-1]
    priorities = distance_matrix[current_node]

    for node in range(1, num_nodes+1):
        if node in tour:
            priorities[node] = float('inf')

    return -priorities

In [198]:
tour, total_distance = tsp(a280.distance_matrix)
total_distance

np.float64(408102.0)

In [191]:
specification = r'''
import numpy as np

def tsp(distance_matrix):
    """greedy algorithm"""
    num_nodes = len(distance_matrix)
    current_node = 1
    tour = [current_node]
    total_distance = 0

    while(len(tour) < num_nodes):
        priorities = priority(distance_matrix, tour)
        next_node = int(np.argmax(priorities) + 1)
        total_distance -= priorities[next_node]
        tour.append(next_node)

    total_distance += distance_matrix[tour[-1]][1]
    return tour, total_distance

@funsearch.run
def evaluate(distance_matrix):
    """evaluate tsp function by total distance"""
    tour, total_distance = tsp(distance_matrix)
    return -total_distance

@funsearch.evolve
def priority(distance_matrix, tour):
    """return priorities for next node with distance"""
    num_nodes = len(distance_matrix)
    current_node = tour[-1]
    priorities = distance_matrix[current_node]

    for node in range(1, num_nodes+1):
        if node in tour:
            priorities[node] = float('inf')

    return -priorities
'''

## 5-Main

In [19]:
# !pip install torch
# !pip install tensorboard

In [20]:
from implementation import funsearch
from implementation import config

In [21]:
# It should be noted that the if __name__ == '__main__' is required.
# Because the inner code uses multiprocess evaluation.
if __name__ == '__main__':
    class_config = config.ClassConfig(llm_class=LLMAPI, sandbox_class=Sandbox)
    main_config = config.Config(samples_per_prompt=4, evaluate_timeout_seconds=30)
    global_max_sample_num = 10  # if it is set to None, funsearch will execute an endless loop
    funsearch.main(
        specification=specification,
        inputs=dataset,
        config=main_config,
        max_sample_nums=global_max_sample_num,
        class_config=class_config,
        log_dir='../logs/funsearch_llm_api'
    )

INFO:absl:Best score of island 0 increased to -866528.6
INFO:absl:Best score of island 1 increased to -866528.6
INFO:absl:Best score of island 2 increased to -866528.6
INFO:absl:Best score of island 3 increased to -866528.6
INFO:absl:Best score of island 4 increased to -866528.6
INFO:absl:Best score of island 5 increased to -866528.6
INFO:absl:Best score of island 6 increased to -866528.6
INFO:absl:Best score of island 7 increased to -866528.6
INFO:absl:Best score of island 8 increased to -866528.6
INFO:absl:Best score of island 9 increased to -866528.6


================= Evaluated Function =================
def priority(num_nodes, tour, distance_matrix):
    current_node = tour[-1]
    priorities = -distance_matrix[current_node]
    
    for node in range(num_nodes):
        if node in tour: 
            priorities[node] = float('inf')
    
    return priorities
------------------------------------------------------
Score        : -866528.6
Sample time  : None
Evaluate time: 3.973773241043091
Sample orders: None


================= Evaluated Function =================
def priority(num_nodes, tour, distance_matrix):
    current_node = tour[-1]
    priorities = -distance_matrix[current_node]
    node_scores = {}

    for node in range(num_nodes):
        if node in tour:
            priorities[node] = float('inf')
        else:
            # Calculate a score based on distance and additional heuristics
            distance_score = -distance_matrix[current_node][node]
            heuristic_score = 1 / (1 + len(tour) - tour.index(node))  